In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

import plotly.express as px
import plotly.graph_objects as go

# Reduce the dimensionality of the embedded series using PCA
from sklearn.decomposition import PCA


In [2]:
# list_data_name = ["BTCUSDT_1d", "ETHUSDT_1d", "BNBUSDT_1d", "XRPUSDT_1d", "ADAUSDT_1d", "LTCUSDT_1d"]                   
# list_data_name = ["BTCUSDT_1h", "ETHUSDT_1h", "BNBUSDT_1h", "XRPUSDT_1h", "ADAUSDT_1h", "LTCUSDT_1h"]                 
  
# list_data_name = ["ADAUSDT_1h"]                   
list_data_name = ["ADAUSDT_1d"]                   
                        
from_date = '2021-12-31 00:00:00' # 2020-12-31 00:00:00
until_date = '2025-07-31 00:00:00'
column_name = 'processed_log_return_wtmra_0'
list_delay_in_days = [1, 3, 7, 14, 21, 30]
dimension = 3  # Adjust as needed

In [3]:
# Lets implement time delay embedding

def time_delay_embedding(series, delay, dimension):
    embedded_data = np.array([series[i: i + delay * dimension: delay] for i in range(len(series) - delay * (dimension - 1))])
    return embedded_data

In [4]:
for data_name in list_data_name:
    
    # Importing data
    data = pd.read_csv(f'../data/01-output-{data_name}-from-{from_date}-until-{until_date}-log-return.csv', parse_dates=['date'], index_col='date')
    data = data[[column_name]]
    
    for delay in list_delay_in_days:
        # print(delay)
        embedded_series = time_delay_embedding(data[column_name], delay * 24, dimension)
        # print(f'embedded_series.shape: {embedded_series.shape}')
        # print(embedded_series)
        pca = PCA(n_components=1)
        pca.fit(embedded_series)
        reduced_series = pca.transform(embedded_series)
        # print(f'reduced_series.shape: {reduced_series.shape}')
        # print(type(reduced_series))
        # add in the beggining of reduced_series the dimension NaN values
        reduced_series_new = np.concatenate((np.full(((dimension - 1) * delay * 24, 1), np.nan), reduced_series), axis=0)
        # print(f'reduced_series_new.shape: {reduced_series_new.shape}')
        data[f'time_delay_{delay}_in_days_dimension_{dimension}_dimension_embedding_pca_1'] = reduced_series_new    

        # remove the nan values
        data = data.dropna()

        # Exporting data
        data.to_csv(f'../data/02e-output-{data_name}-log-return-time-delay-embedding.csv')        

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.